In [1]:
import numpy as np
from scipy.special import factorial
import re
import itertools
from enum import Enum

import sys
sys.path.append('/Users/andrew/Desktop/sudoku/src/sudoku')

from board import Board
from solutions import Solutions
import utils
from grid_string import GridString, read_solutions_file


In [2]:
puzzles = read_solutions_file('/Users/andrew/Desktop/sudoku/data/solutions6.txt')

In [28]:
Rotations = Enum('Rotation', '0 90 180 270')

class ShuffledGrid:
    
    def __init__(self, parent_grid_string: GridString,
                 labels=None,
                 stacks=None,
                 bands=None,
                 columns=None,
                 rows=None,
                 reflection=False,
                 rotation=None):
        assert parent_grid_string.is_seed
        
        self.parent_grid_string = parent_grid_string
        self.labels = labels
        self.stacks = stacks
        self.bands = bands
        self.columns = columns
        self.rows = rows
        self.reflection = reflection
        self.rotation = rotation
        
        self._grid_string = None
        
    @property
    def grid_string(self):
        if self._grid_string is None:
            self._grid_string = self.parent_grid_string.copy()
            
            if self.labels:
                self.relabel()
        
        return self._grid_string
    
    def mirror(self, other):
        self.labels = other.labels
        self.stacks = other.stacks
        self.bands = other.bands
        self.columns = other.columns
        self.rows = other.rows
        self.reflection = other.reflection
        self.rotation = other.rotation
        self._grid_string = None
        return self
        
    def relabel(self):
        assert type(self.labels) is dict
        self.labels[0] = 0
        assert set(self.labels) == set(self.labels.values()) == set(range(len(self.labels)))
        
        self._grid_string.array = np.vectorize(lambda x: self.labels[x])(self.parent_grid_string.array)

In [29]:
class Shuffler:
    
    def __init__(self, grid_string: GridString):
        self.grid_string = grid_string
        
    def permute_labels(self):
        max_digit = self.grid_string.max_digit
        perm_tuples = itertools.permutations(range(1, max_digit+1), max_digit)
        permutations = [{i+1: tupl[i] for i in range(len(tupl))} for tupl in perm_tuples]
        shuffled_grids = [ShuffledGrid(self.grid_string, perm) for perm in permutations]
        return shuffled_grids

In [31]:
shuffles = {}
for puzzle, solution in puzzles.items():
    if solution and solution.is_seed:
        shuffled_puzzles = Shuffler(puzzle).permute_labels()
        shuffles[puzzle] = {p: ShuffledGrid(solution).mirror(p) for p in shuffled_puzzles}

In [32]:
shuffles

{12343.1.23.14123: {<__main__.ShuffledGrid at 0x837cc7390>: <__main__.ShuffledGrid at 0x837cc7710>,
  <__main__.ShuffledGrid at 0x837cc7dd8>: <__main__.ShuffledGrid at 0x837cb3a58>,
  <__main__.ShuffledGrid at 0x837cc7f98>: <__main__.ShuffledGrid at 0x837cb3f60>,
  <__main__.ShuffledGrid at 0x837cc7470>: <__main__.ShuffledGrid at 0x837cb3dd8>,
  <__main__.ShuffledGrid at 0x837cc72b0>: <__main__.ShuffledGrid at 0x837cb3fd0>,
  <__main__.ShuffledGrid at 0x837cc7278>: <__main__.ShuffledGrid at 0x837cb3f98>,
  <__main__.ShuffledGrid at 0x837cc7438>: <__main__.ShuffledGrid at 0x837cb3b00>,
  <__main__.ShuffledGrid at 0x837cc7ef0>: <__main__.ShuffledGrid at 0x837cb3940>,
  <__main__.ShuffledGrid at 0x837cc7fd0>: <__main__.ShuffledGrid at 0x837cb3748>,
  <__main__.ShuffledGrid at 0x837cc7550>: <__main__.ShuffledGrid at 0x837cb3828>,
  <__main__.ShuffledGrid at 0x837cc7400>: <__main__.ShuffledGrid at 0x837cb38d0>,
  <__main__.ShuffledGrid at 0x837cc7f60>: <__main__.ShuffledGrid at 0x837cc60b8>

In [27]:
for h, w in dimensions:
    print("top_boxes_seq({0}, {1}) = {2}".format(h, w, top_boxes_seq(h, w)))

top_boxes_seq(2, 2) = 4
top_boxes_seq(2, 3) = 36
top_boxes_seq(2, 4) = 576
top_boxes_seq(3, 2) = 128
top_boxes_seq(3, 3) = 93312
top_boxes_seq(3, 4) = 382205952
top_boxes_seq(4, 2) = 12288
top_boxes_seq(4, 3) = 6530347008
top_boxes_seq(4, 4) = 109561042308169728


In [32]:
8*np.prod([factorial(i) for i in (4,2,2,2,2)])

3072.0

In [33]:
8*np.prod([factorial(i) for i in (6,2,3,3,2)])

829440.0

In [34]:
8*np.prod([factorial(i) for i in (9,3,3,3,3)])

3762339840.0

In [36]:
6670903752021072936960/5472730538

1218935174261.0999

In [37]:
factorial(9)*factorial(3)*factorial(3)*3*factorial(3)*3*factorial(3)*4*2

33861058560.0

In [39]:
'2341223'.find('9')

-1

In [5]:
grid_string = '2.2.12343.1.23.14123'
dot_index = grid_string.find('.')
dot_index2 = dot_index + 1 + grid_string[dot_index + 1:].find('.')
r = int(grid_string[:dot_index])
c = int(grid_string[dot_index+1:dot_index2])
grid = grid_string[dot_index2+1:]

In [6]:
print(dot_index)
print(dot_index2)

1
3


In [15]:
print(r)
print(c)
print(grid)

2
2
12343.1.23.14123


In [4]:
''.join

'range(0, 10)'

In [6]:
False == None

False

In [6]:
range(5)

range(0, 5)

In [4]:
p.array

array([[0, 2, 3, 4],
       [4, 0, 1, 0],
       [0, 0, 0, 1],
       [2, 0, 0, 0]])

In [5]:
a = p.copy()

In [6]:
a.array

array([[0, 2, 3, 4],
       [4, 0, 1, 0],
       [0, 0, 0, 1],
       [2, 0, 0, 0]])

In [11]:
m = {0:0, 1:2, 2:3, 3:4, 4:1}
f = np.vectorize(lambda x: m[x])
f(a.array)

array([[0, 3, 4, 1],
       [1, 0, 2, 0],
       [0, 0, 0, 2],
       [3, 0, 0, 0]])

In [17]:
re.sub('[^0-9,\.]', '', np.array_str(a.array).replace('0', '.'))

'.2344.1....12...'

In [14]:
np.array_str(a.array).replace('0', '.')

'[[. 2 3 4]\n [4 . 1 .]\n [. . . 1]\n [2 . . .]]'

In [9]:
x = np.arange(9).reshape((3,3))**2
np.where(x < 5, x, -x)

array([[  0,   1,   4],
       [ -9, -16, -25],
       [-36, -49, -64]])

In [12]:
type(m) == dict

True

In [39]:
sg.grid_string

.3411.2....23...

In [38]:
sg = ShuffledGrid(a, labels={1:2, 2:3, 3:4, 4:1})

In [37]:
np.vectorize(lambda x: sg.labels[x])(sg.parent_grid_string.array)

array([[0, 3, 4, 1],
       [1, 0, 2, 0],
       [0, 0, 0, 2],
       [3, 0, 0, 0]])

In [48]:
max_digit = 16
all_digits_string = ''.join([str(i) for i in range(1, max_digit+1)])
permutations = [''.join(lst) for lst in itertools.permutations(all_digits_string, max_digit)]


KeyboardInterrupt: 

In [47]:
[{i+1: lst[i] for i in range(len(lst))} for lst in itertools.permutations(all_digits_string, max_digit)]

[{1: '1', 2: '2', 3: '3', 4: '4'},
 {1: '1', 2: '2', 3: '4', 4: '3'},
 {1: '1', 2: '3', 3: '2', 4: '4'},
 {1: '1', 2: '3', 3: '4', 4: '2'},
 {1: '1', 2: '4', 3: '2', 4: '3'},
 {1: '1', 2: '4', 3: '3', 4: '2'},
 {1: '2', 2: '1', 3: '3', 4: '4'},
 {1: '2', 2: '1', 3: '4', 4: '3'},
 {1: '2', 2: '3', 3: '1', 4: '4'},
 {1: '2', 2: '3', 3: '4', 4: '1'},
 {1: '2', 2: '4', 3: '1', 4: '3'},
 {1: '2', 2: '4', 3: '3', 4: '1'},
 {1: '3', 2: '1', 3: '2', 4: '4'},
 {1: '3', 2: '1', 3: '4', 4: '2'},
 {1: '3', 2: '2', 3: '1', 4: '4'},
 {1: '3', 2: '2', 3: '4', 4: '1'},
 {1: '3', 2: '4', 3: '1', 4: '2'},
 {1: '3', 2: '4', 3: '2', 4: '1'},
 {1: '4', 2: '1', 3: '2', 4: '3'},
 {1: '4', 2: '1', 3: '3', 4: '2'},
 {1: '4', 2: '2', 3: '1', 4: '3'},
 {1: '4', 2: '2', 3: '3', 4: '1'},
 {1: '4', 2: '3', 3: '1', 4: '2'},
 {1: '4', 2: '3', 3: '2', 4: '1'}]

In [59]:
max_digit = 10
perm_tuples = itertools.permutations(range(1, max_digit+1), max_digit)
permutations = [{i+1: tupl[i] for i in range(len(tupl))} for tupl in perm_tuples]

[{1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10},
 {1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 10, 10: 9},
 {1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 9, 9: 8, 10: 10},
 {1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 9, 9: 10, 10: 8},
 {1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 10, 9: 8, 10: 9},
 {1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 10, 9: 9, 10: 8},
 {1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 8, 8: 7, 9: 9, 10: 10},
 {1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 8, 8: 7, 9: 10, 10: 9},
 {1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 8, 8: 9, 9: 7, 10: 10},
 {1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 8, 8: 9, 9: 10, 10: 7},
 {1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 8, 8: 10, 9: 7, 10: 9},
 {1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 8, 8: 10, 9: 9, 10: 7},
 {1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 9, 8: 7, 9: 8, 10: 10},
 {1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 9, 8: 7, 9: 10, 10: 8},
 {1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 9, 8: 8, 9: 7, 10: 10},
 {1: 1, 2: 2, 3: 3, 4: 4,

In [61]:
len(perm_tuples)

TypeError: object of type 'itertools.permutations' has no len()

In [ ]:
permutations = [''.join(lst) for lst in itertools.permutations(all_digits_string, max_digit)]
